In [ ]:
import requests
import json
from tqdm import tqdm
import pandas as pd
import os

In [28]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
api_url = "https://api.barq.social/graphql"
authorization = os.getenv("barq_authorization")
headers = {
    "Authorization": "Bearer " + authorization,
    'Content-type': 'application/json'
}

In [63]:
# Get users
all_profiles = []
offset = 0
increment = 60

body = {
	"operationName": "ProfileOverview",
	"variables": {
		"offset": 0,
		"limit": 60,
		"search": {
			"isAd": False,
			"location": {
				"placeId": "832",
				"latitude": 40.7306,
				"longitude": -73.9866,
				"type": "distance"
			}
		},
		"isAd": False
	},
	"query": "query ProfileOverview($isAd: Boolean, $search: ProfileSearchInput!, $offset: Int = 0, $limit: Int = 30) {\n  profiles(search: $search, offset: $offset, limit: $limit) {\n    uuid\n    displayName\n    relationType\n    images(isAd: $isAd) {\n      image {\n        ...UploadedImage\n        __typename\n      }\n      __typename\n    }\n    location {\n      type\n      distance\n      searchDistance\n      place {\n        place\n        countryCode\n        __typename\n      }\n      __typename\n    }\n    roles\n    __typename\n  }\n}\n\nfragment UploadedImage on UploadedImage {\n  uuid\n  isExplicit\n  contentRating\n  __typename\n}\n"
}



In [64]:
stopped = False

with tqdm(total=50000) as pbar:
    while not stopped:
        res = requests.post(api_url, data=json.dumps(body), headers=headers)
        raw_json = json.loads(res.text)

        if 'data' in raw_json:
            offset += increment
            body['variables']['offset'] = offset

            profiles = raw_json['data']['profiles']
            all_profiles += profiles

            with open('all_profiles.json', 'w', encoding='utf-8') as file:
                json.dump(all_profiles, file)
        else:
            stopped = True
            
        pbar.update(increment)

  1%|          | 360/50000 [00:07<18:05, 45.72it/s]


KeyboardInterrupt: 

In [ ]:
all_profiles

In [65]:
# Get detailed users info
all_detailed_users = []

body = {
	"operationName": "ProfileDetail",
	"variables": {
		"uuid": "d04a15bd-ca07-4543-996f-058f12ac12e7",
		"isAd": False,
		"location": {
			"longitude": -73.9866,
			"latitude": 40.7306
		}
	},
	"query": "query ProfileDetail($uuid: String!, $isAd: Boolean, $location: SearchLocationInput) {\n  profile(uuid: $uuid, location: $location) {\n    uuid\n    displayName\n    relationType\n    isAdOptIn\n    isBirthday\n    age\n    profileImage(isAd: $isAd) {\n      image {\n        ...UploadedImage\n        __typename\n      }\n      __typename\n    }\n    privacySettings {\n      ...PrivacyFragment\n      __typename\n    }\n    images {\n      id\n      image {\n        ...UploadedImage\n        __typename\n      }\n      isAd\n      likeCount\n      hasLiked\n      accessPermission\n      __typename\n    }\n    location {\n      type\n      distance\n      place {\n        place\n        region\n        countryCode\n        longitude\n        latitude\n        __typename\n      }\n      __typename\n    }\n    bio {\n      biography\n      genders\n      languages\n      relationshipStatus\n      sexualOrientation\n      interests\n      hobbies {\n        id\n        interest\n        __typename\n      }\n      socialAccounts {\n        ...SocialAccountsFragment\n        __typename\n      }\n      __typename\n    }\n    bioAd {\n      biography\n      sexPositions\n      behaviour\n      safeSex\n      canHost\n      __typename\n    }\n    sonas {\n      id\n      displayName\n      images {\n        id\n        __typename\n      }\n      description\n      hasFursuit\n      species {\n        id\n        displayName\n        __typename\n      }\n      __typename\n    }\n    kinks {\n      kink {\n        id\n        displayName\n        categoryName\n        isVerified\n        isSinglePlayer\n        __typename\n      }\n      pleasureReceive\n      pleasureGive\n      __typename\n    }\n    groups {\n      group {\n        uuid\n        displayName\n        isAd\n        contentRating\n        image {\n          ...UploadedImage\n          __typename\n        }\n        __typename\n      }\n      threadCount\n      replyCount\n      __typename\n    }\n    events {\n      event {\n        uuid\n        displayName\n        isAd\n        contentRating\n        eventBeginAt\n        eventEndAt\n        image {\n          ...UploadedImage\n          __typename\n        }\n        __typename\n      }\n      isWaitingList\n      __typename\n    }\n    roles\n    shareHash\n    __typename\n  }\n}\n\nfragment UploadedImage on UploadedImage {\n  uuid\n  isExplicit\n  contentRating\n  __typename\n}\n\nfragment PrivacyFragment on PrivacySettings {\n  startChat\n  viewAge\n  viewAd\n  viewKinks\n  viewProfile\n  blockAdults\n  blockMinors\n  __typename\n}\n\nfragment SocialAccountsFragment on ProfileSocialAccounts {\n  twitter {\n    value\n    accessPermission\n    __typename\n  }\n  twitterAd {\n    value\n    accessPermission\n    __typename\n  }\n  telegram {\n    value\n    accessPermission\n    __typename\n  }\n  instagram {\n    value\n    accessPermission\n    __typename\n  }\n  steam {\n    value\n    accessPermission\n    __typename\n  }\n  discord {\n    value\n    accessPermission\n    __typename\n  }\n  deviantArt {\n    value\n    accessPermission\n    __typename\n  }\n  furAffinity {\n    value\n    accessPermission\n    __typename\n  }\n  __typename\n}\n"
}

In [69]:
stopped = False

for profile in tqdm(all_profiles):
    body['variables']['uuid'] = profile['uuid']
    res = requests.post(api_url, data=json.dumps(body), headers=headers)
    raw_json = json.loads(res.text)

    detailed_profile = raw_json['data']['profile']

    all_detailed_users.append(detailed_profile)

    with open('all_detailed_profiles.json', 'w', encoding='utf-8') as file:
        json.dump(all_detailed_users, file)

  2%|▏         | 7/360 [00:03<03:21,  1.75it/s]


KeyboardInterrupt: 